In [ ]:
!pip install yfinance==0.2.44
!pip install crewai==0.70.1
!pip install crewai_tools==0.12.1
!pip install langchain==0.2.16
!pip install langchain_community==0.2.17
!pip install langchain_openai==0.1.25
!pip install duckduckgo_search==6.3.0
!pip install pysqlite3-binary


In [33]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [29]:
import json
import os
from datetime import datetime, timedelta

from crewai import Agent, Task, Crew, Process

from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults

from crewai_tools import CSVSearchTool

import yfinance as yf

from IPython.display import Markdown

In [34]:
from dotenv import load_dotenv

load_dotenv()

openaiApiKey = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(model='gpt-3.5-turbo')

In [6]:
csvWalletTool = CSVSearchTool('/home/flavioaraujo/AI/Ai-Agent/1730725640338-attachment.csv')

In [35]:
customerManager = Agent(
    role="Customer Stocks Manager",
    goal="Get the customer question about the stock {ticket} and search the customer wallet CSV file for the stocks.",
    backstory="""You are the manager of the customer investiments wallet.
    You are the client first contact and you provide the other analystis with the necessary stock and wallet information 
    """,
    verbose=True,
    llm=llm,
    max_iter=5, # quantas tentativas
    tools=[csvWalletTool],
    allow_delegation=False,
    memory=True
)

In [8]:
getCustomerWallet = Task(
    description="""Use the customer question and find the {ticket} in the CSV File.
     Provide if the stock is in the customer wallet and if it is, provide with the mean price he paid and the total numbers of the stocks onwed.""",
     expected_output="If the customer owns the stocks, provide the mean price paid and the total stock numbers.",
     agent=customerManager
)

In [9]:
stocketPriceAnalyst = Agent(
    role="Senior Stock Price Analyst",
    goal="Find the {ticket} stock price and analyses price trends. compare with the price that customer paid.",
    backstory="""You are a highly experienced in analyzing the price of specific stocks and make predictions about its future price""",
    verbose=True,
    llm=llm,
    max_iter=5, # quantas tentativas
    allow_delegation=False,
    memory=True
)

In [14]:
# CRIANDO YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)
    stock = yf.download(ticket, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return stock

yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from the Yahoo Finance API",
    func= lambda ticket: fetch_stock_price(ticket)
)

In [15]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output="""Specify the current trend stocks price - UP, down or sideway.
    eg. stock= 'AAPL, price UP'""",
    tools=[yahoo_finance_tool],
    agent=stocketPriceAnalyst
)

In [16]:
newsAnalyst = Agent(
    role="News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company.
    Provide a market Fear & Greed Index score about the company. For each requested stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed""",
    backstory="""You are highly experienced in analyzing market trends and news for more then 10 years.
    You are also a master level analyst in the human psychology.
    
    You understand the news, their titlee and information, but you look at those with a health dose of skeptcism.
    You consider the source of the news articles""",
    verbose=True,
    llm=llm,
    max_iter=5, # quantas tentativas
    allow_delegation=False,
    memory=True
)

In [18]:
searchTool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [19]:
getNews = Task(
    description=f"""Use the search tool to search news about the stock ticket.
    The current date is {datetime.now()}7
    Compose the result into a helpfull report.""",
    expected_output="""A summary of the overall market and one paragraph summary for the requested asset.
    Include the fear/greed score based on the news. Use format:
    <STOCK TICKET>
    <SUMMARY BASEF ON NEWS>
    <FEAR/GREED SCORE>""",
    agent=newsAnalyst,
    tool=[searchTool]
)

In [20]:
stockRecommender = Agent(
    role="Chief Stock Analyst",
    goal="Get the data from the customer currently stocks, the provided input of stock price trends and the stock news to provide a recommendation: Buy, Sell or Hold the stock.",
    backstory="""You are the leader of the stock analyst team, you have a great performance in the past 20 years in stock recommendation.
    With all your team informations, you are able to provide the best recommendation for the customer to achieve the maximum value creation""",
        verbose=True,
    llm=llm,
    max_iter=5, # quantas tentativas
    allow_delegation=True, # Permite caso ele queira ele devolve a tarefa para algum agente anterior
    memory=True)

In [21]:
recommendStock = Task(
    description="""Use the stock price trend, the stock news report and the customers stock mean price of the {ticket} to provide a recommendation: Buy, Sell or Hold. If the previous reports are not well provided you can delegate back to the specific analyst to work again in the their task""",
    expected_output="""A brief paragraph with the summary of the reasons for recommendation and the recommendation it self in one of the three possible outputs: Buy, Sell or Hold. Use the format:
    <SUMMARY OF REASONS>
    <RECOMMENDATION> """,
    agent=stockRecommender,
    context=[getCustomerWallet, getStockPrice, getNews]
)

In [24]:
copyWriter = Agent(
    role="Stock Content Writer",
    goal="""Write an insghtfull compelling and informative 6 paragraph long newsletter based on the stock price report, the news report and the recommendation report""",
    backstory="""You are a unbeliveble copy writer that understand complex finance concepts and explain for a dummie audience.
    You create complelling stories and narratives that resonate with the audience.""",
    verbose=True,
    llm=llm,
    max_iter=5, # quantas tentativas
    allow_delegation=False,
    memory=True
)

In [25]:
writeNewsletter = Task(
    description="""Use the stock price trend, the stock news report and the stock recommendation to write an insghtfull compelling and informative 6 paragraph long newsletter.
    Focus on the stock price trend, the news, the fear/greed score and the summary reason for the recommendation.
    Include the recommendation in the newsletter""",
    expected_output="""An eloquent 6 paragraph newsletter formated as Markdown in an easy readable manner. it should contain:
    - Introduction - set the overal picture
    - Main part - providdes the meat of the analysis including the stock price trend, the news, the fear/greed score and the summary reason for the recommendation
    - 3 bullets of the main summary reason of the recommendation
    - Recommendation Summary
    - Recommendation it self """,
    agent=copyWriter,
    context=[getStockPrice, getNews, recommendStock]

)

In [26]:
crew = Crew(
    agents=[customerManager, stocketPriceAnalyst, newsAnalyst, stockRecommender, copyWriter],
    tasks=[getCustomerWallet, getStockPrice,getNews,recommendStock,writeNewsletter],
    verbose=True,
    process=Process.hierarchical,
    full_output= True,
    share_crew=False,
    manager_llm=llm,
    max_iter= 15
)

In [ ]:
results = crew.kickoff(inputs={'ticket': "Give your thoughts about Amazon stocks"})

In [30]:
Markdown(results.raw)

Dear Valued Readers,

Welcome to this week's newsletter focusing on Amazon stock analysis. Let's delve into the latest updates and insights regarding AMZN.

Starting with the stock price trend, Amazon (AMZN) has shown resilience in the market with a steady upward trajectory. Despite recent market volatility, AMZN has maintained its position as a strong performer, reflecting the company's solid fundamentals and investor confidence.

In recent news, Amazon has announced plans to further expand its technological capabilities with investments in AI and cloud services. These strategic moves are expected to enhance Amazon's competitive edge in the ever-evolving tech industry and fuel future growth potential.

As for the fear/greed score, sentiment towards Amazon remains largely positive, with a healthy balance between caution and optimism among investors. This indicates a stable market outlook for AMZN and suggests a favorable environment for potential growth.

According to our Senior Stock Price Analyst, the recommendation for Amazon stock (AMZN) is a strong buy. With a bullish outlook on the company's performance and growth prospects, the analyst believes that AMZN presents a lucrative opportunity for investors seeking long-term value and returns.

In conclusion, Amazon continues to showcase its resilience and innovation in the market, making it a compelling choice for investors looking to capitalize on the e-commerce giant's continued success. Stay tuned for more updates on Amazon and other market insights in our future newsletters.

Thank you for your continued readership and trust in our analysis.

Warm regards,
[Your Name]
Stock Content Writer